<a href="https://colab.research.google.com/github/lucarenz1997/NLP/blob/main/Stage_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage 3: Implementing an RAG System for Question Answering

Part 1: Model Training Steps
Objective: Developing and utilizing advanced embedding models to represent the content of Cleantech Media and Google Patent datasets and compare domain-specific embeddings to gain unique insights.

Output: Notebook with annotated model training steps

Data Preparation for Embeddings
Lead: Alvaro Cervan

Preprocessing Steps
The preprocessing steps have already been completed in the previous stage, which include:

Dropping duplicates
Setting data types
Dropping unnecessary columns
Tokenizing text data
Stopword Removal
Language detection
Translating non-English text to English
Lemmatization
These steps were applied to both datasets, media and patents, and the resulting data was saved in the data folder. We will now load the data and perform the following steps:

## SETUP & DATA LOADING

Installationen (einmalig):

In [1]:
!pip install transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [12]:
!pip install langchain langchain-community langchain-openai chromadb --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 63.9 MB/s eta 0:00

In [2]:
# Import imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tqdm import tqdm
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### LOAD & PREPARE DATA

In [3]:
prepdata_media= pd.read_csv("/content/drive/MyDrive/CLT/data/processed_media_data_backup.csv")
prepdata_patent = pd.read_csv("/content/drive/MyDrive/CLT/data/processed_patent_data_backup.csv")
print("Media Backup:")
prepdata_media.head(5)

Media Backup:


,Unnamed: 0,title,date,author,content,domain,url,processed_text
0,93320,"XPeng Delivered ~100,000 Vehicles In 2021",2022-01-02,Unknown,['Chinese automotive startup XPeng has shown o...,cleantechnica,https://cleantechnica.com/2022/01/02/xpeng-del...,chinese automotive startup XPeng show one dram...
1,93321,Green Hydrogen: Drop In Bucket Or Big Splash?,2022-01-02,Unknown,['Sinopec has laid plans to build the largest ...,cleantechnica,https://cleantechnica.com/2022/01/02/its-a-gre...,Sinopec lay plan build large green hydrogen pr...
2,98159,World’ s largest floating PV plant goes online...,2022-01-03,Unknown,['Huaneng Power International has switched on ...,pv-magazine,https://www.pv-magazine.com/2022/01/03/worlds-...,Huaneng Power International switch MW float pv...
3,98158,Iran wants to deploy 10 GW of renewables over ...,2022-01-03,Unknown,"['According to the Iranian authorities, there ...",pv-magazine,https://www.pv-magazine.com/2022/01/03/iran-wa...,accord iranian authority currently renewable e...
4,31128,Eastern Interconnection Power Grid Said ‘ Bein...,2022-01-03,Unknown,['Sign in to get the best natural gas news and...,naturalgasintel,https://www.naturalgasintel.com/eastern-interc...,sign get good natural gas news datum follow to...


In [4]:
prepdata_patent.head(5)

,publication_number,application_number,title,abstract,publication_date,inventor,processed_text
0,US-2022239235-A1,US-202217717397-A,Adaptable DC-AC Inverter Drive System and Oper...,Disclosed is an adaptable DC-AC inverter syste...,2022-07-28 00:00:00,[],disclose adaptable DC AC inverter system opera...
1,US-2022239251-A1,US-202217580956-A,System for providing the energy from a single ...,"In accordance with an example embodiment, a so...",2022-07-28 00:00:00,[],in accordance example embodiment solar energy ...
2,EP-4033090-A1,EP-21152924-A,Method for controlling a wind energy system,Verfahren zum Steuern einer Windenergieanlage ...,2022-07-27 00:00:00,"['Schaper, Ulf', 'von Aswege, Enno', 'Gerke Fu...",Verfahren zum steuern einer Windenergieanlage ...
3,EP-4033090-A1,EP-21152924-A,Method for controlling a wind energy system,Verfahren zum Steuern einer Windenergieanlage ...,2022-07-27 00:00:00,"['Schaper, Ulf', 'von Aswege, Enno', 'Gerke Fu...",Verfahren zum steuern einer Windenergieanlage ...
4,US-11396827-B2,US-202117606042-A,Control method for optimizing solar-to-power e...,A control method for optimizing a solar-to-pow...,2022-07-26 00:00:00,[],a control method optimize solar power efficien...


## Select 50–100 Relevant Paragraphs

In [ ]:
# Function to select long, unique paragraphs
#def get_paragraphs(df, min_words=40, max_paragraphs=50):
#    paragraphs = df["processed_text"].dropna().unique()
#    filtered = [p for p in paragraphs if len(p.split()) >= min_words]
#    return filtered[:max_paragraphs]
#
## Extract paragraphs
#media_paragraphs = get_paragraphs(prepdata_media, max_paragraphs=50)
#patent_paragraphs = get_paragraphs(prepdata_patent, max_paragraphs=50)
#
## Combine and export to CSV for manual processing
#selected_paragraphs = media_paragraphs + patent_paragraphs
#
#pd.DataFrame({'paragraph': selected_paragraphs}).to_csv(
#    "/content/drive/MyDrive/CLT/data/selected_paragraphs.csv", index=False
#)

# Load generated dataset with relevant paragraphs

In [ ]:
#selected_data= pd.read_csv("/content/drive/MyDrive/CLT/data/Stage_3/selected_paragraphs.csv")

### Load manually Generate QA Pairs in ChatGPT-4

In [9]:
#Load manually Generate QA Pairs in ChatGPT-4
evaluation_data= pd.read_csv("/content/drive/MyDrive/CLT/data/Stage_3/Classified_QA_Pairs_cat.csv")
evaluation_data.head(10)

,Paragraph,Question,Answer,Index,Category
0,chinese automotive startup XPeng show one dram...,How has XPeng's vehicle delivery performance e...,XPeng experienced a dramatic increase in vehic...,1,Factual Questions
1,chinese automotive startup XPeng show one dram...,What infrastructure growth has XPeng achieved ...,XPeng has rapidly expanded its infrastructure ...,2,Factual Questions
2,chinese automotive startup XPeng show one dram...,How does XPeng's growth compare to other elect...,"While XPeng is not directly compared to Tesla,...",3,Comparative Questions
3,Sinopec lay plan build large green hydrogen pr...,What is Sinopec's role in the green hydrogen s...,Sinopec is actively entering the green hydroge...,4,Factual Questions
4,Sinopec lay plan build large green hydrogen pr...,How does Sinopec plan to power its green hydro...,Sinopec's project will be powered by a newly b...,5,Strategic or Predictive Questions
5,Sinopec lay plan build large green hydrogen pr...,Why is Sinopec’s investment in green hydrogen ...,Sinopec’s investment is significant because it...,6,Analytical Questions
6,Huaneng Power International switch MW float pv...,What is significant about Huaneng Power Intern...,Huaneng Power International has completed the ...,7,Factual Questions
7,Huaneng Power International switch MW float pv...,What makes the Qinggang Photovoltaic Power Sta...,"The Qinggang Photovoltaic Power Station, launc...",8,Factual Questions
8,Huaneng Power International switch MW float pv...,How is Huaneng Power contributing to innovativ...,Huaneng Power is also developing a 2 GW solar ...,9,Factual Questions
9,accord iranian authority currently renewable e...,What steps is Iran taking to expand its renewa...,"Iran plans to add 10,000 MW of renewable energ...",10,Factual Questions


## RAG

### Chunks

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def get_recursive_splitter(chunk_size: int, chunk_overlap: int):
    return RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", "(?<=\. )", " ", ""],
        length_function=len,
    )

splitter = get_recursive_splitter(512, 64)

def prepare_documents(df: pd.DataFrame):
    docs = []
    for _, row in df.iterrows():
        content = row["processed_text"]
        if pd.isna(content): continue
        chunks = splitter.split_text(content)
        for i, chunk in enumerate(chunks):
            docs.append(Document(page_content=chunk, metadata={"source": row.get("title", "N/A")}))
    return docs

media_docs = prepare_documents(prepdata_media)
patent_docs = prepare_documents(prepdata_patent)
all_docs = media_docs + patent_docs
print(f"Total chunks: {len(all_docs)}")

Total chunks: 214167


### Vektordatenbank + Retriever aufbauen

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vectorstore = Chroma.from_documents(
    documents=all_docs,
    embedding=embedding_model,
    persist_directory="/content/chroma_eval_db"
)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})


<ipython-input-13-4754efacd9e8>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### RAG-Kette bauen (Frage → Kontext → Antwort)

In [ ]:
llm = ChatOpenAI(model="gpt-4")

def create_rag_chain(retriever):
    template = """You are an assistant for question-answering. Use the following context to answer the question. If unsure, say so.

Question: {question}
Context: {context}
Answer:"""

    prompt = ChatPromptTemplate.from_template(template)

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    chain = RunnableParallel({
        "context": retriever,
        "question": RunnablePassthrough()
    }).assign(
        answer=(RunnablePassthrough.assign(context=lambda x: format_docs(x["context"])) | prompt | llm | StrOutputParser())
    )
    return chain

rag_chain = create_rag_chain(retriever)


### QA-Paare evaluieren lassen

In [ ]:
from ragas import evaluate
from ragas.metrics import answer_relevancy, context_relevancy, faithfulness

from datasets import Dataset

# Format in RAGAS-kompatible Struktur
def format_for_ragas(df: pd.DataFrame):
    return Dataset.from_dict({
        "question": df["Question"],
        "ground_truth": df["Answer"],
    })

# Generiere Antworten für alle Fragen
results = []
for _, row in tqdm(evaluation_data.iterrows(), total=len(evaluation_data)):
    question = row["Question"]
    generated = rag_chain.invoke(question)
    results.append({
        "question": question,
        "ground_truth": row["Answer"],
        "generated_answer": generated["answer"]
    })

qa_dataset = Dataset.from_pandas(pd.DataFrame(results))

# Evaluation mit RAGAS
eval_result = evaluate(
    qa_dataset,
    metrics=[
        answer_relevancy,
        context_relevancy,
        faithfulness
    ]
)

eval_result.to_pandas()


### Durchschnittswerte pro Fragekategorie berechnen

In [ ]:
# Kombinieren mit Kategorien
eval_df = eval_result.to_pandas()
eval_df["Category"] = evaluation_data["Category"]

category_scores = eval_df.groupby("Category")[
    ["answer_relevancy", "context_relevancy", "faithfulness"]
].mean().round(3)

print(category_scores)
